In [1]:
import os
import numpy as np
import pandas as pd
import fnmatch
import re
from itertools import groupby
from Bio import SeqIO

In [2]:
# concatenate like files

def concat_files(path1, path2, split_char, split_position, new_file_ending, input_ext):
    # path1 = current file path
    # path2 = where the new file will go
    # split_char = what to split by, ex. '_'
    # split_position = position to split filename, ex 0
    # new_file_ending = ending to add after the split, ex "-merged.fa"
    # input_ext = file extension of input, ex = ".fasta"
    
    files = fnmatch.filter(os.listdir(path1), "*" + input_ext)
    
    def split_name(fname): return fname.split(split_char)[split_position]
    
    for name, folder in groupby(sorted(files, key=split_name), key=split_name):
        new_name = str(name) + new_file_ending
        with open(os.path.join(path2, new_name), 'w') as outfile:
            for fname in folder:
                with open(os.path.join(path1, fname), 'r') as infile:
                    outfile.write(infile.read())

In [3]:
path = "/scicomp/home-pure/top8/lepto/lepto_genomes/primers"
merge_path = "/scicomp/home-pure/top8/lepto/lepto_genomes/primers/merged_primers"
large_serovar_path = "/scicomp/home-pure/top8/lepto/lepto_genomes/primers/large_serovar_concat/"
bacterio_path = "/scicomp/home-pure/top8/lepto/lepto_genomes/primers/bacterio_serovars"


# concatenate forward and reverse primers that are of same species epithet
# concat_files(path, merge_path, '_', -1, "_concat.fna", '.fasta')

# concatenate all concat files from above to find alignments of entire genome
# bash was used
# for f in *_concat.fna; do cat $f >> large_serovar_concat/large_serovar_concat.fna

# concatenate forward and reverse primers of same species epithet from bacterio.net
# concat_files(bacterio_path, bacterio_path, '.', 0, "_concat.fa", ".fna")


In [4]:
# find alignment and other information of each file
# all mafft files have _mafft.txt ending

def alignment(path_name, which_format):
    # path_name = path to files
    # which_format = merged, individual, or bacterio
    
    # initialize variables
    names = []
    alignments = []
    sequence_num = []
    accessions = []
    lengths = []
    name = ''
    records = []

    # select each file
    for file in fnmatch.filter(os.listdir(path_name), "*" + '.txt'):
        alignment = 0
        matches = 0
        mismatches = 0

        if which_format == 'serovar':
            # get name to process concat file
            fasta_name_list = file.split('_')[:-2]
            fasta_name_list = [name + "_" for name in fasta_name_list]
            fasta_name = ''.join(map(str, fasta_name_list))

            # use SeqIO to save info about the sequence file
            records = list(SeqIO.parse(os.path.join(path_name, fasta_name + 'concat.fna'), "fasta"))
        
        elif which_format == 'merged':
            # get name to process concat file
            name = file.split('-')[0]
            fasta_name = file.split('_')[0]

            # use SeqIO to save info about the sequence file
            records = list(SeqIO.parse(os.path.join(path_name,fasta_name), "fasta"))
        
        elif which_format == 'individual':
            # get name to process concat file
            fasta_name_list = file.split('_')[:-1]
            fasta_name_list = [name + "_" for name in fasta_name_list]
            fasta_name = ''.join(map(str, fasta_name_list))

            # get accession number
            acc = re.search(r'.+\((GCF_.+)\)_.+', file)
            accessions.append(acc.group(1))
            
            # use SeqIO to save info about the sequence file
            records = list(SeqIO.parse(os.path.join(path_name, fasta_name + 'concat.fna'), "fasta"))
        
        elif which_format == 'bacterio':
            # get name to process concat file
            fasta_name = file.split('_')[0]

            # use SeqIO to save info about the sequence file
            records = list(SeqIO.parse(os.path.join(path_name, fasta_name + '_concat.fa'), "fasta"))

            # bacterio.net files are <numbers>.fna.
            # inside files have 'Leptospira_<epithet>_...fna' and the epithet needs to be extracted
            get_name = list(SeqIO.parse(os.path.join(path_name, fasta_name + '.fna'), 'fasta'))
            for record in get_name:
                split_id = record.id
                split_id = split_id.split('_')
                name = split_id[1]
                accessions.append(split_id[3]) # not actually accession - used to make life easier

        else:
            break

        sequence_num.append(len(records)) # number of sequences

        # get length of each sequence in the file
        if which_format != 'serovar':
            tmp_lengths = []
            for record in records:
                tmp_lengths.append(len(record.seq))

            lengths.append(tmp_lengths)

        # proccess mafft alignment files
        with open(os.path.join(path_name, file)) as mafft_file:
            next(mafft_file) # skip first line
            for line in mafft_file:
                if line.startswith("NZ_"):
                    continue
                else:
                    matches += line.count("*")
                    mismatches += line.count(".")
        if matches != 0:
            alignment = (matches/(matches+mismatches))*100
        else:
            alignment = 0
        names.append(name)
        alignments.append(alignment)
        
    return names, alignments, sequence_num, accessions, lengths


In [5]:
# make alignments of merged data, each refseq file, and large_serovar data files
merge_names, merge_alignments, merge_sequence_num, merge_acc, merge_lengths = alignment(merge_path, 'merged')
ind_names, ind_alignments, ind_sequence_num, ind_acc, ind_lengths = alignment(path, 'individual')
serovar_names, serovar_alignments, serovar_sequence_num, serovar_acc, serovar_lengths = alignment(large_serovar_path, 'serovar')

In [6]:
#merged data
merged_data = {'species':merge_names, '# of seq':merge_sequence_num, 'nuc. identity':merge_alignments}
merged = pd.DataFrame(merged_data)
print(merged.head())

           species  # of seq  nuc. identity
0      abararensis         0       0.000000
1           adleri         3      97.385621
2     ainazelensis         1     100.000000
3  ainlahdjerensis         1     100.000000
4       alexanderi         6     100.000000


In [7]:
# large serovar data
large_serovar_data = {'# of seq':serovar_sequence_num, 'nuc. identity':serovar_alignments}
large_serovar = pd.DataFrame(large_serovar_data)
print(large_serovar.head())

   # of seq  nuc. identity
0       466       71.33758


In [8]:
#percent length per AE010300 (153 bp)
def find_perc_lengths(which_lengths):
    percent_lengths = []
    for i in which_lengths:
        tmp_perc_lengths = []
        for j in i:
            tmp_perc_lengths.append(round((j/153)*100, 2))
            
        percent_lengths.append(tmp_perc_lengths)

    return percent_lengths

In [9]:
#ind data
ind_percent_lengths = find_perc_lengths(ind_lengths)
ind_data = {'accession':ind_acc, '# of seq':ind_sequence_num, 'nuc. identity':ind_alignments, 'length per AE010300':ind_percent_lengths}
ind = pd.DataFrame(ind_data)
print(ind.head())

         accession  # of seq  nuc. identity length per AE010300
0  GCF_016918735.1         0            0.0                  []
1  GCF_004770415.1         1          100.0             [100.0]
2  GCF_002811845.1         1          100.0             [100.0]
3  GCF_002811985.1         1          100.0             [100.0]
4  GCF_016918785.1         1          100.0             [100.0]


In [10]:
# genome_list.tsv
df = pd.read_csv('/scicomp/home-pure/top8/lepto/lepto_genomes/genome_list.tsv', sep="\t")
df = df[['assembly_accession', 'organism_name', 'infraspecific_name']]
df.rename({'assembly_accession':'accession', 'organism_name':'species_org', 'infraspecific_name':'strain'}, axis=1, inplace=True)

# replace strings and work on serovar data
df['species_org'] = df['species_org'].str.replace("Leptospira", "")
df['strain'] = df['strain'].str.replace('strain=', '')
df['serovar'] = df['species_org'].str.contains('serovar')
df['serovar'] = df['serovar'].map({True: 'serovar', False: ''})
move_serovar = df.pop('serovar')
df.insert(2, 'serovar', move_serovar)

# work on species data
df['species'] = df['species_org'].str.split(' ').str[1]
move_species = df.pop('species')
df.insert(1, 'species', move_species)
remove_species_org = df.pop('species_org')
print(df.head())

         accession  species serovar     strain
0  GCF_004769835.1  biflexa          201601962
1  GCF_004770315.1  biflexa          201601961
2  GCF_004770325.1  biflexa          201601960
3  GCF_004770345.1  biflexa          201601958
4  GCF_004770805.1  biflexa          201601959


In [11]:
# merge genome_list and each acc data
df2 = pd.merge(df, ind)
print(df2.head())

         accession  species serovar     strain  # of seq  nuc. identity  \
0  GCF_004769835.1  biflexa          201601962         0            0.0   
1  GCF_004770315.1  biflexa          201601961         0            0.0   
2  GCF_004770325.1  biflexa          201601960         0            0.0   
3  GCF_004770345.1  biflexa          201601958         0            0.0   
4  GCF_004770805.1  biflexa          201601959         0            0.0   

  length per AE010300  
0                  []  
1                  []  
2                  []  
3                  []  
4                  []  


In [12]:
# remove brackets from percent length per AE10300 column
for c in df2.columns:
    df2[c]= df2.apply(lambda x: str(x[c]).replace("[",'').replace(']',''), axis=1)

df2['length per AE010300'] = df2['length per AE010300'].replace('', 0)
print(df2.head())

         accession  species serovar     strain # of seq nuc. identity  \
0  GCF_004769835.1  biflexa          201601962        0           0.0   
1  GCF_004770315.1  biflexa          201601961        0           0.0   
2  GCF_004770325.1  biflexa          201601960        0           0.0   
3  GCF_004770345.1  biflexa          201601958        0           0.0   
4  GCF_004770805.1  biflexa          201601959        0           0.0   

  length per AE010300  
0                   0  
1                   0  
2                   0  
3                   0  
4                   0  


In [13]:
# send to tsv file
df2.to_csv('genome_by_accession.tsv', sep="\t", index=False)
merged.to_csv('genome_by_species.tsv', sep="\t", index=False)

In [14]:
# count number of sequences found in merged data
zeros = 0
less_than_5 = 0 
greater_than_10 = 0
for i in merge_sequence_num:
    if i == 0:
        zeros += 1
    elif 0 < i >= 5:
        less_than_5 += 1
    else:
        greater_than_10 += 1

# data to use to create bar graph in excel
print('zeros:\t', zeros, '\n<5:\t', less_than_5, '\n>10:\t', greater_than_10)

zeros:	 29 
<5:	 14 
>10:	 26


In [15]:
# Obtain accessions where row == 'serovar'
serovar_df = df2.copy()
rows_to_drop = serovar_df[serovar_df['serovar'] != 'serovar'].index
serovar_df.drop(rows_to_drop, inplace=True)
serovar_acc = serovar_df['accession']
serovar_acc.to_csv('serovar_accession.tsv', sep="\t", index=False)

# Use bash to concatenate files from tsv file above
# while IFS= read -r file; do cat *\("$file"\)_concat.fna >> serovar_concat.fna; done < "../serovar_accession.tsv"

In [16]:
# use csv file of approved species from SME
approved_species = pd.read_csv('/scicomp/home-pure/top8/lepto/lepto_genomes/approved_species.csv', sep=",")
approved_species = approved_species[['Name', 'Clade']]
approved_species.rename({'Clade':'clade'})
# approved_species.to_csv('check_names.tsv', sep='\t', index=False)
approved_species['species'] = approved_species['Name'].astype(str).str.split().str[1]
# approved_species.drop('Name', axis=1, inplace=True)
approved_species.to_csv('check_names.tsv', sep='\t', index=False)

print(approved_species.head())
# approved_species.groupby(['species']).size()

                                           Name Clade          species
0        Leptospira abarensis Korba et al. 2021    S1        abarensis
1        Leptospira adleri Thibeaux et al. 2020    P1           adleri
2     Leptospira ainazelensis Korba et al. 2021    P1     ainazelensis
3  Leptospira ainlahdjerensis Korba et al. 2021    P1  ainlahdjerensis
4     Leptospira alexanderi Brenner et al. 1999    P1       alexanderi


In [17]:
# work on bacterio.net data
bacterio_names, bacterio_alignments, bacterio_sequence_num, bacterio_acc, bacterio_lengths = alignment(bacterio_path, 'bacterio')

bacterio_perc_lengths = find_perc_lengths(bacterio_lengths)

bacterio_data = {"species":bacterio_names, "# of seq":bacterio_sequence_num, "nuc. identity":bacterio_alignments, "length per AE10300":bacterio_perc_lengths}
bacterio_df = pd.DataFrame(bacterio_data)
bacterio_df['species'] = bacterio_df['species'].str.replace('abararensis', 'abarensis') # misspelling of word found
print(bacterio_df.head())

          species  # of seq  nuc. identity length per AE10300
0  ellinghausenii         0            0.0                 []
1       johnsonii         1          100.0            [100.0]
2        ryugenii         0            0.0                 []
3          adleri         1          100.0            [100.0]
4      barantonii         1          100.0            [100.0]


In [18]:
bacterio_merge = pd.merge(bacterio_df, approved_species, how="outer")
bacterio_merge = bacterio_merge.groupby("species").first().reset_index()

# remove brackets from percent lengths
for c in bacterio_merge.columns:
    bacterio_merge[c]= bacterio_merge.apply(lambda x: str(x[c]).replace("[",' ').replace(']',''), axis=1)

bacterio_merge['length per AE10300'] = bacterio_merge['length per AE10300'].str.replace(' ', '0')
bacterio_merge.drop(bacterio_merge[bacterio_merge['species'] == 'Leptospira'].index, inplace=True) # found random Leptospira added
print(bacterio_merge.head())

bacterio_merge.to_csv('approved_serovars.tsv', sep="\t", index=False)

           species # of seq nuc. identity length per AE10300  \
1        abarensis      0.0           0.0                  0   
2           adleri      1.0         100.0             0100.0   
3     ainazelensis      1.0         100.0             0100.0   
4  ainlahdjerensis      1.0         100.0             0100.0   
5       alexanderi      1.0         100.0             0100.0   

                                           Name Clade  
1        Leptospira abarensis Korba et al. 2021    S1  
2        Leptospira adleri Thibeaux et al. 2020    P1  
3     Leptospira ainazelensis Korba et al. 2021    P1  
4  Leptospira ainlahdjerensis Korba et al. 2021    P1  
5     Leptospira alexanderi Brenner et al. 1999    P1  


In [19]:
# merge approved_species to df2 to get clade column
df2_clades = pd.merge(df2, approved_species, how="outer")
print(df2_clades.head())

         accession  species serovar     strain # of seq nuc. identity  \
0  GCF_004769835.1  biflexa          201601962        0           0.0   
1  GCF_004770315.1  biflexa          201601961        0           0.0   
2  GCF_004770325.1  biflexa          201601960        0           0.0   
3  GCF_004770345.1  biflexa          201601958        0           0.0   
4  GCF_004770805.1  biflexa          201601959        0           0.0   

  length per AE010300                                               Name Clade  
0                   0  Leptospira biflexa (Wolbach and Binger 1914) N...    S1  
1                   0  Leptospira biflexa (Wolbach and Binger 1914) N...    S1  
2                   0  Leptospira biflexa (Wolbach and Binger 1914) N...    S1  
3                   0  Leptospira biflexa (Wolbach and Binger 1914) N...    S1  
4                   0  Leptospira biflexa (Wolbach and Binger 1914) N...    S1  


In [20]:
# add approved_species to merged to get clade column
merged_clades = pd.merge(merged, approved_species, how="outer")
print(merged_clades.head())

           species  # of seq  nuc. identity  \
0      abararensis       0.0       0.000000   
1           adleri       3.0      97.385621   
2     ainazelensis       1.0     100.000000   
3  ainlahdjerensis       1.0     100.000000   
4       alexanderi       6.0     100.000000   

                                           Name Clade  
0                                           NaN   NaN  
1        Leptospira adleri Thibeaux et al. 2020    P1  
2     Leptospira ainazelensis Korba et al. 2021    P1  
3  Leptospira ainlahdjerensis Korba et al. 2021    P1  
4     Leptospira alexanderi Brenner et al. 1999    P1  


In [21]:
# export to tsv file
df2_clades.to_csv("accession_with_clade.tsv", sep="\t", index=False)
merged_clades.to_csv("species_with_clade.tsv", sep="\t", index=False)